# Getting Started with Selene

This tutorial explores the core components of Selene, and should teach you everything you need to know to train a simple model on biological sequence data. 
Before starting this tutorial, you need to install Selene.
Instructions for installation are available [here](https://selene.flatironinstitute.org/installation.html).
Lastly, if you are not familiar with neural networks, we recommend reading through this [introductory PyTorch tutorial on neural networks](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html).
In the simplest case, we train a neural network as follows:

1. Construct our neural network, which should be a [`torch.nn.Module`](https://pytorch.org/docs/stable/nn .html#torch.nn.Module) object
2. Load the training data and divide it into training and validation sets
3. Iterate over the training set
4. Compute and backpropagate the training loss after each iteration
5. Save the model weights at specified intervals
6. Compute and report the loss on the validation set at specified intervals
7. Compute and report the loss on the validation set after training is complete

In this case, much of our work is already done for us.
In fact, we do not actually need to write any code besides our model and a configuration file.

## Download and format the data

First, we need to download the data. (We provide a "Shortcut" below that contains all the formatted data in a `.tar.gz`.)

In this tutorial, we can go through a single-feature example with [data](http://hgdownload.cse.ucsc.edu/goldenpath/hg19/encodeDCC/wgEncodeAwgTfbsUniform/) from the ENCODE Uniform TFBS composite track. This is the transcription factor dataset that was used to train [Zhou and Troyanskaya's (2015)](https://doi.org/10.1038/nmeth.3547) DeepSEA model.

We can download the measurements for transcription factor CTCF in cell type GM12878 by running

```sh
wget http://hgdownload.cse.ucsc.edu/goldenpath/hg19/encodeDCC/wgEncodeAwgTfbsUniform/wgEncodeAwgTfbsUtaGm12878CtcfUniPk.narrowPeak.gz
```

and format the data with

```sh
bgzip -d wgEncodeAwgTfbsUtaGm12878CtcfUniPk.narrowPeak.gz

cut -f 1-3 wgEncodeAwgTfbsUtaGm12878CtcfUniPk.narrowPeak > GM12878_CTCF.bed

sed -i "s/$/\tGM12878|CTCF|None/" GM12878_CTCF.bed

sort -k1V -k2n -k3n GM12878_CTCF.bed > sorted_GM12878_CTCF.bed
```

The formatted BED file should contain 4 columns, in order: chromosome, start, end, feature. We do not support strand-specific data at this time (it will be added in the next version of Selene).

In this example, we will use the `IntervalsSampler` class for partitioning and sampling the data. The intervals sampler requires that we pass in an intervals file with 3 columns: chrom, start, end. This intervals file determines where in the genome we sample our data. We have provided an intervals file for you with the regions in the original DeepSEA dataset that contained at least 1 transcription factor (TF). We will refer to this file as "DeepSEA TF regions" from now on.

It also requires that we tabix-index the dataset BED file for fast querying of targets in genomic regions.

```sh
bgzip -c sorted_GM12878_CTCF.bed > sorted_GM12878_CTCF.bed.gz

tabix -p bed sorted_GM12878_CTCF.bed.gz
```

Selene provides sampler classes to partition your dataset into training/testing/validation sets and will draw examples from the appropriate partitions during the training/evaluation process.

These sampler classes require that you have a file containing the distinct features that the model predicts:
```sh
cut -f 4 sorted_GM12878.bed | sort -u > distinct_features.txt
```

Finally, we must download the hg19 FASTA file:

```sh
wget https://www.encodeproject.org/files/male.hg19/@@download/male.hg19.fasta.gz

bgzip -d male.hg19.fasta.gz
```

### Shortcut: download all formatted data from Zenodo record

Download the compressed data from here:

```sh
wget https://zenodo.org/record/1319886/files/selene_quickstart_tutorial.tar.gz
```

Extract it and `mv` all files from the extracted directory `selene_quickstart_tutorial` to the current directory.

## Command line arguments

Selene uses a limited number (two to be precise) of command line arguments.
The first of these is the positional parameter for the configuration file, which we will discuss in more detail in the following section.
The second argument is the optional named argument for the learning rate, specified with `--lr`.
The learning rate only needs to be specified when Selene is training a model, and is ignored in all other circumstances.

## Configuration file syntax

The configuration file is a [YAML file](https://en.wikipedia.org/wiki/YAML) that specifies the majority of the runtime parameters for Selene.
In general, a YAML file with keys `key1` and `key2` taking values `val1` and `val2` would look like such:

```YAML
---
key1: val1
key2: val2
...
```

For training a new network, there are a few keys that we must include in this YAML file, which we will discuss later.

The following sections explain each of these parameters in some detail.
However, we first need to discuss the syntax for our configuration file.
We discuss each of the argument types for configuration files below.

### Literal arguments

The simplest configuration arguments are literals.
To specify a learning rate of `0.01`, and that we would like to evaluate test metric performance and not save the datasets, we would include the following lines in our configuration file:
```YAML
lr: 0.01
random_seed: 123
output_dir: path/to/output/dir
```

Note that at the top-level, we do not separate these arguments with commas. 

### List arguments

After literals, lists arguments like `ops` are the next simplest type of configuration parameter.
Syntactically, list arguments are very similar to the python lists that they represent.
For instance, to specify `ops` as the Python list below:
```python
ops = ["train"]
```
we would write the following line in our configuration file:
```YAML
ops: [train]
```

### Dictionary arguments

The next type of argument we need is a dictionary.
Like lists, dictionaries in the configuration file are very similar to their Python equivalents.
For instance, if the `model` configuration were written as a dictionary in Python, it might look something like the following:
```python
model = {"file": "/absolute/path/to/deeperdeepsea.py",
         "class": "DeeperDeepSEA",
         "sequence_length": 1000
         "n_classes_to_predict": 1,
         "non_strand_specific": {
             "use_module": True,
             "mode": "mean"
             }
         }
```
Now, to write this in the configuration file, we simply include the following lines:
```YAML
model: {
    file: /absolute/path/to/deepsea.py,
    class: DeeperDeepSEA,
    sequence_length: 1000,
    n_classes_to_predict: 1,
    non_strand_specific: {
        use_module: True,
        mode: mean
    }
}
```

### Function arguments 

In addition to the types we've just discussed, Selene's configuration accept python function calls.
For instance, let's say we want to specify the value of the `features` argument for `train_model`, which takes a list of strings and specifies the names of the values we are predicting with our model.
One option would be to write the list of strings into the configuration file, but this might take a long time if this list is very long.
If we were using Python, we would just read the list of feature names the following:
```python
import selene_sdk
features = selene_sdk.utils.load_features_list(input_path="features_list.txt")
```
Fortunately, we can use function the function call arguments to include this in our configuration file.
Specifically, we would write the following in our configuration file:
```YAML
features: !obj.selene_sdk.utils.load_features_list {
    input_path: "features_list.txt"
}
```

## Training a model and analyzing sequences with it

To train or analyze sequences with a model, we first specify the configuration file and then we execute `selene` from the command line.
The first section provides an overview of all the requirements for training a model with Selene.
The second section covers the arguments used to evaluate sequences with a trained model.
We recommend opening the included `train.yaml` and `analyze.yaml` configuration files and following along in them while reading through these sections.

### Configuration file arguments for training
Before running Selene from the command line, we need to specify its runtime parameters in a configuration file.
Specifically, we need to include the following:

| key              | definition |
|------------------|-----------------------------------------------------------------------------------------------------|
| ops              | list of operations to execute with Selene |
| model            | dict containing the configuration parameters for the model we intend to train.
| sampler          | a subclass of selene.samplers.Sampler |
| train_model      | a subclass of selene.TrainModel |
| lr               | a floating point value for the learning rate, if we do not want to specify it in the command line arguments |
| evaluate_on_test | a boolean specifying whether we should calculate performance metrics on held out test data |
| save_datasets    | a boolean specifying if we would like to write the training/validation/test data to file                   |

#### ops

Selene currently supports the `train` and `analyze` operations, and allows chaining of operations by simply adding them to the `ops` list in the configuration file.
For instance, to train a model and then use it to analyze some data, you would include the following line in the configuration file:
```YAML
ops: [train, analyze]
```
To only train a model, we would just write the following:
```YAML
ops: [train]
```

Each individual operation comes with its own requirements about what keys are in the configuration file. These keys contain the class configurations required to run a given operation. We have examples of configuration files [here](https://github.com/FunctionLab/selene/tree/master/config_examples) for the main operations that you can run with Selene.

#### model

In this tutorial, we will use an example neural network from [DeepSEA](http://deepsea.princeton.edu), which models chromatin properties of sequences in the non-coding genome.
The class for this model, `DeepSEA`, is specified in the `deepsea.py` file from earlier.
The model should follow all of the [normal rules for specifying a `torch.nn.Module`](https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_module.html), with two exceptions.
First, the file with the model class should include a method called `criterion` that returns the object to use for the [PyTorch loss function](https://pytorch.org/docs/master/nn.html#loss-functions).
In `deepsea.py`, this is defined as follows:
```python
def criterion():
    return torch.nn.BCELoss()
```
Second, we must define a method called `get_optimizer` that takes a learning rate, and returns the [optimization function](https://pytorch.org/docs/master/optim.html) and its parameters.
The return value should be a 2-tuple, where the first element is the optimizer class, and the second element is a `dict` containing the keyword arguments to use when constructing the optimizer.
In `deepsea.py`, this is specified as follows:
```python
def get_optimizer(lr):
    return (torch.optim.SGD, {"lr": lr, "weight_decay": 1e-6, "momentum": 0.9})
```
Note that, to allow specifying the learning rate at the command line, you should include the passed `lr` argument in the `dict` of keyword arguments.

#### sampler

The `sampler` argument specifies how Selene will sample its training data.
The value for `sampler` should be a function-type argument, and the function needed to construct an object that is a subclass of `selene.samplers.Sampler`. 
The specific arguments for the sampler's construction will vary by class, so it is important to check the class definitions and documentation when specifying them.
For the example, we will use the following configuration for the `sampler`:
```YAML
sampler: !obj:selene_sdk.samplers.IntervalsSampler {
    reference_sequence: !obj:selene_sdk.sequences.Genome {
        input_path: hg19.fasta
    },
    features: !obj:selene_sdk.utils.load_features_list {
        input_path: distinct_features.txt
    },
    target_path: sorted_GM12878_CTCF.bed.gz
    intervals_path: deepsea_TF_intervals.txt,
    sequence_length: 1000,
    center_bin_to_predict: 200,
    test_holdout: [chr8, chr9],
    validation_holdout: [chr6, chr7],
    feature_thresholds: 0.5,
    seed: 127,
    mode: "train",
    save_datasets: ["test"]
}
```

The intervals sampler samples from regions specified in `intervals_path`. In this case, we provide you with the regions in the DeepSEA dataset that contained at least 1 transcription factor. 

For your own dataset, you might do something similar. Alternatively, you could sample uniformly from all regions in the genome. If you want to same uniformly across the genome, please look at the documentation for [`selene_sdk.samplers.RandomPositionsSampler`](http://selene.flatironinstitute.org/samplers.html).

#### train_model
The `train_model` argument is responsible to specifying many of the parameters for `selene.TrainModel`.
The following parameters for `train_model` are automatically generated, and should not be specified in the configuration file:

|                |
|----------------|
| model          |
|data_sampler    |
|loss_criterion  |
|optimizer_class |
|optimizer_kwargs|

With this in mind, we write the following in our configuration file:
```YAML
train_model: !obj:selene_sdk.TrainModel {
    batch_size: 64,
    # typically the number of steps is much higher
    max_steps: 16000,  
    # the number of mini-batches the model should sample before reporting performance
    report_stats_every_n_steps: 2000,
    n_validation_samples: 32000,
    cpu_n_threads: 32,
    use_cuda: False,
    data_parallel: False
}
```

#### other arguments

There are three additional optional arguments we need when training models: `lr`, `output_dir`, and `create_subdirectory`.
If you do not want to specify the learning rate in the command line arguments, you can specify it in the configuration file.
However, note that Selene will throw an exception and crash if training is specified in the operations `op` and `lr` is not included in the configuration file or specified in the command line arguments.
If we want to specify it in the configuration file, we can include the following lines:
```YAML
lr: 0.01
```

It is recommended that you specify a random seed for `torch`. This is helpful for reproducibility. 
```YAML
random_seed: 123
```
Lastly, we would include the path to an output directory in the configuration file. If the directory does not exist, it is automatically created for you. 
```YAML
output_dir: ./data/outputs
```

Finally, if `create_subdirectory` is True, Selene will create a subdirectory with the current timestamp as the name within `output_directory`. This is so that you can save multiple runs in a single directory:
```YAML
create_subdirectory: True
```

### Running it

Now, it only takes 2 methods to run training in Selene.

In [1]:
%matplotlib inline

from selene_sdk.utils import load_path
from selene_sdk.utils import parse_configs_and_run

First, we load the configuration file as a dictionary. We provide `./simple_train.yml` for you, but you must review this file and the spots labeled `# TODO` before running the subsequent cells. For example, you need to replace the path in `model["file"]` with the absolute path to the model architecture file.

In [2]:
configs = load_path("./simple_train.yml",
                    # TODO: this should be the default. It is always False in our case.
                    instantiate=False)

Second, execute the operations that were specified in `ops`. This configuration parsing function will look for the classes/parameters corresponding to the operations "train" and "evaluate", instantiate those objects, and run the necessary methods.

TODO: execute this function in full and show the outputs in the rendered Jupyter notebook.

In [ ]:
parse_configs_and_run(configs, lr=0.10)

Outputs and logs saved to ./training_outputs
